In [3]:
from databaseClass import DB
import utils
import sql_files
import numpy as np
import pandas as pd

In [4]:
userName = utils.userName
userPass = utils.userPass
dbName = utils.dbName

In [5]:
db = DB(userName = userName, userPass = userPass, dataBaseName = dbName)

In [6]:
def generate_sql(input_sql_file):
    with open(input_sql_file, 'r') as file:
        sql = file.read()
        return sql
    
def build_table(sql_file, table_name):
    '''postgres does not have a create or replace table option.  so we do that here instead'''
    try:
        db.BuildTableFromQuery(generate_sql(sql_file), table_name)
    except:
        db.dropTable(table_name)
        db.BuildTableFromQuery(generate_sql(sql_file), table_name)

In [34]:
df_url_spread.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6571 entries, /boxscores/201510300ATL.html to /boxscores/202101200TOR.html
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   spread_target  6571 non-null   float64
dtypes: float64(1)
memory usage: 102.7+ KB


In [29]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124431 entries, 0 to 124430
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   url                       124431 non-null  object 
 1   gametype                  124431 non-null  object 
 2   date                      124431 non-null  object 
 3   hometeam                  124431 non-null  object 
 4   awayteam                  124431 non-null  object 
 5   player                    124431 non-null  object 
 6   team                      124431 non-null  object 
 7   two_point_shots_made_x    124431 non-null  float64
 8   two_point_shots_x         124431 non-null  float64
 9   three_point_shots_made_x  124431 non-null  float64
 10  three_point_shots_x       124431 non-null  float64
 11  total_makes_x             124431 non-null  float64
 12  total_shots_x             124431 non-null  float64
 13  defensive_rebounds_x      114402 non-null  f

In [30]:
df_final = db.DBtoDF(generate_sql('sql_files/final.sql'))
df_url_spread = df_final[['url', 'spread_target']]
df_url_spread = df_url_spread.groupby('url').mean()

#values are the features we plan to use for modeling
values = ['two_point_shots_made_y','two_point_shots_y',
          'three_point_shots_made_y','three_point_shots_y',
          'total_makes_y','total_shots_y','defensive_rebounds_y',
          'offensive_rebounds_y']

#df is now going to be in the right shape for modeling
df_final = pd.pivot_table(df_final, values=values, 
                                   columns=['player_team_y'],
                                   index='url',
                                  aggfunc=np.mean)
#collapsing the pivot multi index
df_final.columns = df_final.columns.to_series().str.join('_')
df_final.apply(pd.to_numeric)
#df_final.apply(pd.to_numeric)
#pd.set_option('precision', 2)


In [59]:
df_url_spread['spread_target']

url
/boxscores/201510300ATL.html     3.0
/boxscores/201510300CLE.html    10.0
/boxscores/201510300DET.html     4.0
/boxscores/201510310IND.html   -21.0
/boxscores/201510310LAC.html     5.0
                                ... 
/boxscores/202101200IND.html   -12.0
/boxscores/202101200LAC.html    19.0
/boxscores/202101200MIN.html    -1.0
/boxscores/202101200PHI.html     8.0
/boxscores/202101200TOR.html    -9.0
Name: spread_target, Length: 6571, dtype: float64

In [53]:
df_final = df_final.fillna(0)

In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6571 entries, /boxscores/201510300ATL.html to /boxscores/202101200TOR.html
Columns: 13638 entries, defensive_rebounds_y_A. Afflalo_NYK to two_point_shots_y_Z. Williamson_NOP
dtypes: float64(13638)
memory usage: 683.8+ MB


In [54]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6571 entries, /boxscores/201510300ATL.html to /boxscores/202101200TOR.html
Columns: 13638 entries, defensive_rebounds_y_A. Afflalo_NYK to two_point_shots_y_Z. Williamson_NOP
dtypes: float64(13638)
memory usage: 683.8+ MB


In [55]:
from sklearn.preprocessing import StandardScaler
features = values
# Separating out the features
x = df_final
# Separating out the target
y = df_url_spread
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [70]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1000)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)

In [58]:
df_final

,defensive_rebounds_y_A. Afflalo_NYK,defensive_rebounds_y_A. Afflalo_ORL,defensive_rebounds_y_A. Afflalo_SAC,defensive_rebounds_y_A. Aminu_ORL,defensive_rebounds_y_A. Aminu_POR,defensive_rebounds_y_A. Anderson_LAC,defensive_rebounds_y_A. Anderson_WAS,defensive_rebounds_y_A. Bargnani_BRK,defensive_rebounds_y_A. Baynes_BOS,defensive_rebounds_y_A. Baynes_DET,...,two_point_shots_y_Z. La_MIN,two_point_shots_y_Z. Norvell_GSW,two_point_shots_y_Z. Pachulia_DAL,two_point_shots_y_Z. Pachulia_DET,two_point_shots_y_Z. Pachulia_GSW,two_point_shots_y_Z. Qi_HOU,two_point_shots_y_Z. Randolph_MEM,two_point_shots_y_Z. Randolph_SAC,two_point_shots_y_Z. Smith_PHI,two_point_shots_y_Z. Williamson_NOP
url,,,,,,,,,,,,,,,,,,,,,
/boxscores/201510300ATL.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/boxscores/201510300CLE.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/boxscores/201510300DET.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/boxscores/201510310IND.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/boxscores/201510310LAC.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/boxscores/202101200IND.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/boxscores/202101200LAC.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/boxscores/202101200MIN.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#principalDf['target'] = df_url_spread['spread_target'].values

In [71]:
principalDf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-5.12,-4.58,6.43,-0.87,-2.32,-0.39,-0.52,-4.45,9.25,5.30e-01,...,-0.37,-0.73,0.38,-1.29,-1.60,2.14,0.07,-0.78,-1.19,0.07
1,-4.31,-5.15,2.86,-2.59,-1.71,6.39,-4.57,3.28,1.17,-2.52e+00,...,1.22,-0.27,-0.04,-0.18,-0.13,-2.04,-0.63,-0.57,-2.41,-2.05
2,-9.08,-9.05,8.33,-7.45,-6.13,0.12,-3.17,0.26,-1.79,8.39e-01,...,-0.07,0.17,-0.35,-0.62,0.05,-0.48,-0.43,0.41,-0.73,0.83
3,-8.75,-1.28,8.37,-1.06,-1.91,1.61,-5.91,-1.17,-0.79,1.11e+00,...,-0.55,-0.70,0.42,0.85,-0.67,0.09,-0.34,0.09,1.46,0.34
4,-13.51,5.70,3.47,1.77,7.18,-0.81,-1.01,-13.42,-3.61,6.13e+00,...,0.32,-0.52,-0.23,-0.32,2.27,1.43,-0.51,-0.91,0.13,-1.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6566,18.17,-2.07,9.42,-12.73,-3.75,6.19,-9.43,-9.55,-4.16,-1.73e+01,...,-0.38,2.04,3.84,3.47,0.81,1.91,0.89,4.94,-2.17,2.66
6567,20.43,24.72,13.12,-6.44,-28.79,37.41,2.51,-32.10,-14.50,6.51e+00,...,-1.34,-1.84,-2.48,-1.77,2.11,1.94,1.70,1.07,-2.10,0.45
6568,12.61,3.10,14.10,-1.63,2.56,-4.44,-2.91,-4.86,0.42,-1.77e+00,...,0.18,-1.07,-2.14,-2.94,1.20,1.15,0.46,2.69,3.08,-2.08
6569,19.84,-16.54,5.87,14.70,33.73,12.22,21.72,-3.67,13.82,-6.84e-03,...,1.54,0.01,-3.07,4.42,-0.26,-2.12,-0.83,2.41,-3.93,0.19


In [69]:
import xgboost as xgb

In [77]:
data = principalDf
label = df_url_spread['spread_target'].values
dtrain = xgb.DMatrix(data, label=label)

In [78]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'reg:squarederror'}
param['nthread'] = 4
param['eval_metric'] = 'rmse'

In [79]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [81]:
bst.eval()

TypeError: eval() missing 1 required positional argument: 'data'

In [ ]:
bst.save_model('0001.model')